# Intro

This notebook creates the view for additional account fields which Census will use for the sync


## Change History

<style>
  table {margin-left: 0 !important;}
</style>

| Date    | Author | Description |
| :-------- | :------- | :------- | 
|2025-02-12 | Mclain R |  Created Date|

In [2]:
CREATE OR ALTER VIEW v_hubspot_additional_account_fields AS

-- pre-filter hubspot companies
WITH companies AS (
    SELECT DISTINCT
        id AS company_id,
        property_icrm_account_id,
        property_dcrm_account_id
    FROM [silver_lakehouse].[dbo].[hubspot__company]
    WHERE property_primary_unique_id IS NOT NULL
),

-- hubspot field mappings and inner join on companies to return only matches.
icrm_data AS (
    SELECT
        c.company_id,
        CASE
            WHEN a.slp = 'Yes' THEN 'true'
            WHEN a.slp = 'No' THEN 'false'
            ELSE a.slp
        END AS icrm_f_i_slp,
        a.primary_vsc_provider AS icrm_vsc_provider,
        a.floorplan AS icrm_floorplan,
        a.af_retail AS icrm_af_retail,
        a.vsc_vol__3mo AS icrm_vsl_vol_3_mo,
        a.gap_vol__3mo AS icrm_gap_vol_3_mo,
        a.otherfi_vol__3mo AS icrm_other_fi_vol_3_mo,
        a.product_density_3mo AS icrm_prod_density_3_mo,
        a.[31_day_avg__vol] AS icrm_avg_vol,
        a.cur__31_day_vol AS icrm_current_vol,
        TRY_CAST(REPLACE(a.[%_diff_], '%', '') AS FLOAT) / 100 AS icrm_vol_difference,
        a.call_reports AS icrm_call_report,
        TRY_CAST(a.most_recent_call_date AS DATE) AS icrm_last_call_date,
        a.slc_report AS icrm_slc_report,
        TRY_CAST(a.most_recent_slc_date AS DATE) AS icrm_last_slc_date,
        a.product_density AS icrm_prod_density,
        a.floorplan_lost AS icrm_floor_plan_lost,
        a.paid_off_cash_advance AS icrm_paid__off_cash_advance,
        a.incentive AS icrm_incentive,
        a.reinsurance AS icrm_reinsurance,
        a.risk_score AS icrm_risk__score
    FROM [silver_lakehouse].[dbo].[nucleus__icrm_additional_account_fields] a
    INNER JOIN companies c ON a.uniqueaccountid = c.property_icrm_account_id
),

-- hubspot field mappings and inner join on companies to return only matches.
dcrm_data AS (
    SELECT
        c.company_id,
        a.[privileges] AS dcrm_privilrges,
        a.sub_privileges AS dcrm_sub_privilrges
    FROM [silver_lakehouse].[dbo].[nucleus__dcrm_additional_account_fields] a
    INNER JOIN companies c ON a.uniqueaccountid = c.property_dcrm_account_id
)

-- final select which also uses left join to grab all companies and fill in non-matches with null,
-- which will overwrite outdated information to null in hubspot.
SELECT 
    c.company_id,
    
    -- ICRM Fields
    i.icrm_f_i_slp,
    i.icrm_vsc_provider,
    i.icrm_floorplan,
    i.icrm_af_retail,
    i.icrm_vsl_vol_3_mo,
    i.icrm_gap_vol_3_mo,
    i.icrm_other_fi_vol_3_mo,
    i.icrm_prod_density_3_mo,
    i.icrm_avg_vol,
    i.icrm_current_vol,
    i.icrm_vol_difference,
    i.icrm_call_report,
    i.icrm_last_call_date,
    i.icrm_slc_report,
    i.icrm_last_slc_date,
    i.icrm_prod_density,
    i.icrm_floor_plan_lost,
    i.icrm_paid__off_cash_advance,
    i.icrm_incentive,
    i.icrm_reinsurance,
    i.icrm_risk__score,
    
    -- DCRM Fields
    d.dcrm_privilrges,
    d.dcrm_sub_privilrges

FROM companies c
LEFT JOIN icrm_data i ON c.company_id = i.company_id
LEFT JOIN dcrm_data d ON c.company_id = d.company_id

code for both dcrm and icrm combined in one table